<a href="https://colab.research.google.com/github/redhood18/stock_predictor/blob/main/stock_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as p
import matplotlib.pyplot as mp
import tensorflow as tf
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_asianpaint=p.read_csv("drive/MyDrive/stock predictor/stocks/stocks/ASIANPAINT__EQ__NSE__NSE__MINUTE.csv")
df_axisbank=p.read_csv("drive/MyDrive/stock predictor/stocks/stocks/AXISBANK__EQ__NSE__NSE__MINUTE.csv")
df_godrej=p.read_csv("drive/MyDrive/stock predictor/stocks/stocks/GODREJAGRO__EQ__NSE__NSE__MINUTE.csv")
df_unilivr=p.read_csv("drive/MyDrive/stock predictor/stocks/stocks/HINDUNILVR__EQ__NSE__NSE__MINUTE.csv")

In [ ]:
df_asianpaint.head()
df_asianpaint[-1:]

,timestamp,open,high,low,close,volume
370544,2021-01-01 15:29:00+05:30,2769.1,2772.0,2769.1,2770.1,2975.0


In [ ]:
df_axisbank.head()

,timestamp,open,high,low,close,volume
0,2017-01-02 09:15:00+05:30,451.40,452.00,450.60,451.20,29198.0
1,2017-01-02 09:16:00+05:30,451.15,451.15,450.50,450.80,12539.0
2,2017-01-02 09:17:00+05:30,450.50,450.80,449.30,449.30,26833.0
3,2017-01-02 09:18:00+05:30,449.30,450.10,448.95,449.75,12067.0
4,2017-01-02 09:19:00+05:30,449.75,450.30,449.25,449.80,15675.0


In [ ]:
#df_axisbank=df_axisbank.drop("timestamp",axis=1)
#new_data=df_axisbank.drop("high",axis=1)
new_data=new_data.drop("low",axis=1)
new_data.head()

,open,close,volume
0,451.40,451.20,29198.0
1,451.15,450.80,12539.0
2,450.50,449.30,26833.0
3,449.30,449.75,12067.0
4,449.75,449.80,15675.0


In [ ]:
new_data.isnull().sum()

open      138
close     138
volume    138
dtype: int64

In [ ]:
def get_normalised_data(data):
    """
    Normalises the data values using MinMaxScaler from sklearn
    :param data: a DataFrame with columns as  ['index','Open','Close','Volume']
    :return: a DataFrame with normalised value for all the columns except index
    """
    # Initialize a scaler, then apply it to the features
    scaler = MinMaxScaler()
    numerical = ['open','close','volume']
    data[numerical] = scaler.fit_transform(data[numerical])

    return data


In [ ]:
stock_data=get_normalised_data(new_data)
stock_data.head()
stock_data.isnull().sum()
stock_data.fillna(0,inplace=True)

In [ ]:
def unroll(data, sequence_length=24):
    """
    use different windows for testing and training to stop from leak of information in the data
    :param data: data set to be used for unrolling
    :param sequence_length: window length
    :return: data sets with different window.
    """
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    return np.asarray(result)

In [ ]:
def build_basic_model(input_dim, output_dim, return_sequences):
    """
    Builds a basic lstm model
    :param input_dim: input dimension of the model
    :param output_dim: output dimension of the model
    :param return_sequences: return sequence of the model
    :return: a basic lstm model with 3 layers.
    """
    model = Sequential()
    model.add(LSTM(
        input_shape=(None, input_dim),
        units=output_dim,
        return_sequences=return_sequences))

    model.add(Dropout(0.2))

    model.add(LSTM(
        128,
        return_sequences=False))

    model.add(Dropout(0.2))

    model.add(Dense(
        units=1,activation='linear'))

    return model

In [ ]:
def train_test_split_lstm(stocks, prediction_time=1,test_size=7411, unroll_length=50):
    """
        Split the data set into training and testing feature for Long Short Term Memory Model
        :param stocks: whole data set containing ['Open','Close','Volume'] features
        :param prediction_time: no of days
        :param test_data_size: size of test data to be used
        :param unroll_length: how long a window should be used for train test split
        :return: X_train : training sets of feature
        :return: X_test : test sets of feature
        :return: y_train: training sets of label
        :return: y_test: test sets of label
    """
    # training data
    test_data_cut = test_size + unroll_length + 1

    x_train = stocks[0:-prediction_time - test_data_cut].to_numpy()
    y_train = stocks[prediction_time:-test_data_cut]['close'].to_numpy()

    # test data
    x_test = stocks[0 - test_data_cut:-prediction_time].to_numpy()
    y_test = stocks[prediction_time - test_data_cut:]['close'].to_numpy()

    return x_train, x_test, y_train, y_test

In [ ]:
len(new_data)*0.2

74109.2

In [ ]:
#def create_sequence(df,sequence_length):
  #x,y=[],[]
  #for i in range(len(df)-sequence_length):
    #x.append(df[i:(i+sequence_length)])
    #y.append(df[i+sequence_length])
  #return np.array(x),np.array(y)


In [ ]:
x_train,x_test,y_train,y_test=train_test_split_lstm(stock_data)

In [ ]:
x_test.size

22383

In [ ]:
unroll_length = 50
x_train = unroll(x_train, unroll_length)
x_test = unroll(x_test, unroll_length)
y_train = y_train[-x_train.shape[0]:]
y_test = y_test[-x_test.shape[0]:]

In [ ]:
#from sklearn.model_selection import train_test_split
#x=new_data.drop("close",axis=1)
#y=new_data["close"]


In [ ]:
#split_size=int(0.8*len(x))
#x_train,y_train=x[:split_size],y[:split_size]
#x_test,y_test=x[split_size:],y[split_size:]

In [ ]:
#x_train_reshaped=np.expand_dims(x_train,axis=1)
#x_train_reshaped = np.expand_dims(y_train, axis=1)
#y_train_reshaped = tf.convert_to_tensor(y_train_reshaped, dtype=tf.float32)
#x_test_reshaped=np.expand_dims(x_test,axis=1)
#x_test_reshaped=tf.convert_to_tensor(x_test_reshaped,dtype=tf.float32)
#y_test_reshaped=np.expand_dims(y_test,axis=1)
#y_test_reshaped=tf.convert_to_tensor(y_test_reshaped,dtype=tf.float32)

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout,BatchNormalization,Flatten

In [ ]:
#y_train_reshaped.shape

In [ ]:
#model=Sequential()
#model.add(LSTM(64,activation="relu",return_sequences=True))
#model.add(BatchNormalization())
#model.add(LSTM(50,return_sequences=False))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
#model.add(Dense(15,activation="relu"))
#model.add(Dense(1))
#adam=optimizers.Adam(0.001)
#model.compile(loss="mean_squared_error",optimizer=adam)
#es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
#model.fit(x_train_reshaped,y_train_reshaped,epochs=100,batch_size=64,verbose=1,callbacks=[es],validation_split=0.2)
batch_size = 100
epochs = 5
model = build_basic_model(input_dim = x_train.shape[-1],output_dim = unroll_length, return_sequences=True)
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train,
          y_train,
          epochs=1,
          verbose=1,
          validation_split=0.05
         )

10778/10778 [==============================] - 1171s 108ms/step - loss: 5.2939e-04 - val_loss: 2.4982e-05


In [ ]:
x_train.head()

,open,high,low,volume
0,451.40,452.00,450.60,29198.0
1,451.15,451.15,450.50,12539.0
2,450.50,450.80,449.30,26833.0
3,449.30,450.10,448.95,12067.0
4,449.75,450.30,449.25,15675.0


In [ ]:
import os
import datetime
modeldir=os.path.join("drive/MyDrive/stock predictor/models",datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
model_path=modeldir+"-"+".h5"
model.save(model_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow_hub as hub
def load_model(model_path):
  model=tf.keras.models.load_model(model_path,custom_objects={"KerasLayer":hub.KerasLayer})
  return model
loaded_model=load_model(model_path)

In [ ]:
scores=loaded_model.evaluate(x_test,y_test)
scores

232/232 [==============================] - 11s 43ms/step - loss: 1.3485e-05


1.3484787814377341e-05

In [ ]:
from sklearn.metrics import accuracy_score
test_predictions=loaded_model.predict(x_test,verbose=1)
mp.scatter(test_predictions, color='r', label='Predicted')
mp.scatter(y_test, color='b', label='Actual')
mp.xlabel('X')
mp.ylabel('y')
mp.title('Predicted vs Actual')
mp.legend()
mp.show()

232/232 [==============================] - 8s 35ms/step


TypeError: scatter() missing 1 required positional argument: 'y'